In [ ]:
def store_two_waveforms_first_fit(
    waveform_items,
    M=3,  # number of memories
    N=16384,  # capacity of each memory
):
    """
    We have a sequence of items, where each item is a tuple of 2 waveforms (same length).
    Example item: (9000, 9000) => two waveforms, each length 9000.

    We want to store them using a First-Fit approach:
      - For each chunk of the tuple not yet placed,
        we scan from memory 0 up to M-1 to find the first memory
        that can accommodate 2 * chunk_size samples.

    We return:
      - 'mem_usage': array of length M (how many samples used in each memory)
      - 'placements': for each item, a list [wave0_list, wave1_list]
          * wave0_list: [ (mem_idx, chunk_size), (mem_idx, chunk_size), ... ]
          * wave1_list: [ (mem_idx, chunk_size), (mem_idx, chunk_size), ... ]

    Each (mem_idx, chunk_size) means: "chunk_size samples of that waveform
    were placed in memory mem_idx." The two waveforms in a tuple
    always have identical chunk splits, but we provide separate lists
    so you can see each waveform's distribution.

    :param waveform_items: A list of tuples (S, S), each representing two waveforms
                           of length S (same length).
    :param M: number of memories available (e.g., between 2 and 6).
    :param N: capacity of each memory in samples (e.g. 16384).
    """

    # Tracks how many samples are used in each memory
    memory_usage = [0] * M

    # placements[item_index] = [
    #    [(mem_idx, chunk_size), (mem_idx, chunk_size), ...],  # for waveform #0
    #    [(mem_idx, chunk_size), (mem_idx, chunk_size), ...],  # for waveform #1
    # ]
    placements = []

    # Process each tuple (two waveforms)
    for item in waveform_items:
        # Each item is (S, S). So the length of each waveform is:
        wave_length = item[0]  # second is also item[1], same length

        # We'll store separate sub-lists for each waveform in this tuple
        wave_info = [[], []]  # wave_info[0] => placements for waveform #0
        # wave_info[1] => placements for waveform #1

        start_in_item = 0  # how many samples we've placed so far

        # While there's still something left to store
        while start_in_item < wave_length:
            remaining = wave_length - start_in_item
            placed_chunk = False

            # First-Fit: try memories from 0..M-1
            for mem_idx in range(M):
                # How much space is left in memory mem_idx?
                leftover_capacity = N - memory_usage[mem_idx]
                if leftover_capacity <= 0:
                    # This memory is full, try the next one
                    continue

                # Because we have 2 waveforms in the tuple,
                # each chunk of size chunk_size uses 2 * chunk_size capacity.
                max_chunk = leftover_capacity // 2
                if max_chunk == 0:
                    # Not enough space even for 1 sample of each waveform
                    continue

                # We can place up to max_chunk samples for each waveform,
                # but we only need 'remaining' to finish this tuple.
                chunk_size = min(max_chunk, remaining)

                # Consume the space in mem_idx
                memory_usage[mem_idx] += 2 * chunk_size

                # Record the chunk for each waveform in the tuple
                wave_info[0].append((mem_idx, chunk_size))
                wave_info[1].append((mem_idx, chunk_size))

                # Advance
                start_in_item += chunk_size
                placed_chunk = True
                break  # we placed a chunk => go back to "while" loop

            if not placed_chunk:
                # Could not place any portion => out of space, discard remainder
                raise MemoryError("Not enough memory")

        placements.append(wave_info)

    return {"mem_usage": memory_usage, "placements": placements}

In [ ]:
M = 6
N = 16384

# Suppose each item is a tuple of 2 waveforms, e.g. (S, S).
# For example:
items = [
    (8000, 8000),  # 2 waveforms, each length 8000
    (9000, 9000),  # 2 waveforms, each length 9000
    (16000, 16000),  # 2 waveforms, each length 16000
    (15000, 15000),  # 2 waveforms, each length 15000
]

result = store_two_waveforms_first_fit(items, M, N)
print("Memory usage:", result["mem_usage"])
print("\nPlacements details:")
for i, wave_info in enumerate(result["placements"]):
    wave0_list, wave1_list = wave_info  # each is a list of (mem_idx, chunk_size) pairs
    print(f"Item {i}:")
    print(f"  Waveform 0 => {wave0_list}")
    print(f"  Waveform 1 => {wave1_list}")

items = [
    (45000, 45000),  # 2 waveforms, each length 45000
]

result = store_two_waveforms_first_fit(items, M, N)
print("Memory usage:", result["mem_usage"])
print("\nPlacements details:")
for i, wave_info in enumerate(result["placements"]):
    wave0_list, wave1_list = wave_info  # each is a list of (mem_idx, chunk_size) pairs
    print(f"Item {i}:")
    print(f"  Waveform 0 => {wave0_list}")
    print(f"  Waveform 1 => {wave1_list}")